In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import sys
sys.path.insert(1, 'C:/Users/pnknv/Documents/GitHub/yadisk_loader')
from main_utils import *

Необходимо создать алгоритм обновления существующих и добавления несуществующих файлов в realty </br>
В prices необходимо добавлять все пришедшие цены


# вопросы 
### 0. ЧТО НУЖНО ДОБАВИТЬ
 <b>ad_id в realty и houses в bigint</b> </br> 
 <b>добавить колонку с названиями городов из авито из ссылки</b> </br> 
 <b>добавить индекс к ad_id realty</b> </br>
 <b>добавлять темповую таблицу для обновления realty</b> </br>

### 1. Колонки 
Какие из колонок могут измениться? </br> 
Ключ: ад_айди + ссылка, все остальные на новые значения, STATUS = ? STATUS_NEW=? house_id=?
### 2. Обновление тест базы
Как часто происходи?

In [3]:
st_time = datetime.now()
print('Скрипт запущен: ', get_today_date())

Скрипт запущен:  11/04/2023, 16:50:09


In [4]:
ssh_host, ssh_port, ssh_username, ssh_password, \
database_username, database_password, database_name, \
localhost, localhost_port, table_name, ya_token, ya_api, ya_link = get_config()

In [5]:
local_save_dir_avito = create_load_save_dir('avito')
local_save_dir_cian = create_load_save_dir('cian')

In [6]:
sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost,
                                        localhost_port, database_username, database_password, database_name)

In [7]:
handled_files_avito = get_saved_files_names('avito')
handled_files_cian = get_saved_files_names('cian')

In [8]:
files_to_process_avito = ['27-03-23 (без дублей).csv']
files_to_process_cian = ['циан 10-04-23 (без дублей).csv']

#### функции для новой таблицы

In [9]:
create_table_query = \
    """CREATE TABLE `temp_realty` (
      `id` int(11) NOT NULL AUTO_INCREMENT,
      `source_id` int(11) DEFAULT NULL,
      `ad_id` BIGINT DEFAULT NULL,
      `city_id` int(11) DEFAULT NULL,
      `district_id` int(11) DEFAULT NULL,
      `type_id` int(11) DEFAULT NULL,
      `addr` varchar(255) DEFAULT NULL,
      `square` float DEFAULT '0',
      `floor` int(11) DEFAULT NULL,
      `house_floors` int(11) DEFAULT NULL,
      `link` varchar(512) DEFAULT NULL,
      `date` timestamp NULL DEFAULT CURRENT_TIMESTAMP,
      `status` int(11) DEFAULT '0',
      `version` int(11) DEFAULT '0',
      `offer_from` varchar(255) DEFAULT NULL,
      `status_new` int(1) NOT NULL DEFAULT '0',
      `house_id` int(11) DEFAULT NULL,
      PRIMARY KEY (`id`)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 AVG_ROW_LENGTH=2048 ROW_FORMAT=DYNAMIC;"""

index_show_query = \
f"""SHOW indexes FROM temp_realty"""

index_create_query = \
f"""CREATE INDEX index_ad_id_temp
    ON temp_realty (ad_id);"""

clear_temp_table_query = \
    """DELETE FROM temp_realty"""

In [10]:
def get_exist_ad_id(engine, source):
    source_id = 2 if source == 'cian' else 3
    ad_id_query = f'SELECT ad_id FROM realty where source_id = {source_id}'
    try:
        con_obj = engine.connect()
        ad_id_db = pd.read_sql(text(ad_id_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(traceback.format_exc())
        ad_id_db = None
        exc_code = exc.code
    return ad_id_db, exc_code

In [11]:
exist_ad_id, exc_code = get_exist_ad_id(sql_engine, 'cian')

In [12]:
exist_ad_id

,ad_id
0,272396626
1,272395650
2,272391397
3,272380144
4,272380142
...,...
700867,286003009
700868,286003012
700869,286003015
700870,286003019


In [13]:
def get_date_from_name(fname):
    try:
        return datetime.strptime(fname.replace("(без дублей)", "").replace("циан", "").replace(" ", "")[:8], "%d-%m-%y")
    except:
        return datetime.strptime(Path(fname).stem.replace("(без дублей)", "").replace("циан", "").replace(" ", "")[:8],
                                 "%d-%m-%y")

In [14]:
def get_index_temp(engine):
    index_show_query = \
        f"""SHOW indexes FROM temp_realty"""
    try:
        con_obj = engine.connect()
        index_db = pd.read_sql(text(index_show_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(traceback.format_exc())
        index_db = None
        exc_code = exc.code
    return index_db, exc_code

In [15]:
con_obj = sql_engine.connect()
try:
    con_obj.execute(text(create_table_query))
    if 'index_ad_id_temp' in get_index_temp(sql_engine)[0].Key_name.to_list():
        pass
    else:
        con_obj.execute(text(index_create_query))
    con_obj.commit()
    print('Временная таблица создана')
except:
    con_obj.execute(text(clear_temp_table_query))
    con_obj.commit()
    print('Временная таблица очищена')
con_obj.close()

Временная таблица очищена


In [24]:
dima_c = pd.read_csv('27-03-23.csv', delimiter=',', encoding='cp1251')

In [25]:
dima_c

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
0,cian,210197059,Краснодарский край; Анапа; улица Горького; 2А,Анапа,NaN,NaN,107.0,7,12,3 комн,https://anapa.cian.ru/sale/flat/210197059/,24000000,224299,Собственник,2019-06-16 19:18:17
1,cian,213575526,Краснодарский край; Анапа; улица Краснозеленых...,Анапа,NaN,NaN,106.0,6,7,3 комн,https://anapa.cian.ru/sale/flat/213575526/,10100000,95283,Риелтор,2019-07-26 11:52:55
2,cian,232012292,Краснодарский край; Анапа; улица Ленина; 194,Анапа,NaN,NaN,20.0,1,9,Студия,https://anapa.cian.ru/sale/flat/232012292/,2100000,105000,Риелтор,2020-05-14 15:01:50
3,cian,233965925,Краснодарский край; Анапа; улица Толстого; 81,Анапа,NaN,NaN,29.0,1,3,1 комн,https://anapa.cian.ru/sale/flat/233965925/,4500000,155172,Агентство недвижимости,2020-06-08 14:04:52
4,cian,239179225,Краснодарский край; Анапа; Владимирская улица;...,Анапа,NaN,NaN,140.0,13,13,5 комн,https://anapa.cian.ru/sale/flat/239179225/,15300000,109286,Собственник,2020-08-17 16:40:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103599,cian,285370017,Томская область; Томск; р-н Кировский; д. Лоск...,Томск,Кировский,NaN,57.1,2,4,2 комн,https://tomsk.cian.ru/sale/flat/285370017/,2700000,47285,NaN,2023-03-27 12:54:22
103600,cian,285370096,Томская область; Томск; р-н Октябрьский; мкр. ...,Томск,Октябрьский,Телецентр,53.0,8,10,2 комн,https://tomsk.cian.ru/sale/flat/285370096/,7450000,140566,Агентство недвижимости,2023-03-27 12:55:16
103601,cian,285370463,Томская область; Томск; р-н Ленинский; мкр. Ра...,Томск,Ленинский,Радужный,53.0,6,10,2 комн,https://tomsk.cian.ru/sale/flat/285370463/,4750000,89623,Агентство недвижимости,2023-03-27 12:59:27
103602,cian,285373005,Томская область; Томск; р-н Ленинский; Черемош...,Томск,Ленинский,NaN,31.5,7,10,1 комн,https://tomsk.cian.ru/sale/flat/285373005/,2800000,88889,Агентство недвижимости,2023-03-27 13:24:21


In [29]:
df_cian_realty.head(5)

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,price,price_sqrm
0,2,210197059,5.0,NaN,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-27,0,287,Собственник,1,24000000,224299
1,2,213575526,5.0,NaN,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-27,0,287,Риелтор,1,10100000,95283
2,2,232012292,5.0,NaN,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-27,0,287,Риелтор,1,2100000,105000
3,2,233965925,5.0,NaN,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-27,0,287,Агентство недвижимости,1,4500000,155172
4,2,239179225,5.0,NaN,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-27,0,287,Собственник,1,15300000,109286


In [22]:
ilya_c = pd.read_csv(files_to_process_cian[0], delimiter=';', encoding='utf8')

In [23]:
ilya_c

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Адрес,Дата публикации,Тип продавца,Продавец
0,https://kazan.cian.ru/sale/flat/279841013/,"1-комн. кв., 41 м², 1/20 этаж",1-комн. кв.,41 м²,1/20 этаж,6790000,165610,ЖК «Комос на Губкина»,"Республика Татарстан, Казань, р-н Советский, м...","вчера (8 апр, 18:32)",Агентство недвижимости,Ключ
1,https://kazan.cian.ru/sale/flat/279841011/,"1-комн. кв., 40 м², 10/16 этаж",1-комн. кв.,40 м²,10/16 этаж,6800000,170000,ЖК «Яратам»,"Республика Татарстан, Казань, р-н Советский, м...","вчера (вчера, 09:27)",Агентство недвижимости,Ключ
2,https://kazan.cian.ru/sale/flat/279830163/,"2-комн. кв., 53,5 м², 9/9 этаж",2-комн. кв.,"53,5 м²",9/9 этаж,6700000,125234,NaN,"Республика Татарстан, Казань, р-н Приволжский,...","пять дней назад (4 апр, 15:45)",Агентство недвижимости,Домино
3,https://kazan.cian.ru/sale/flat/279806940/,"1-комн. кв., 35,9 м², 6/17 этаж",1-комн. кв.,"35,9 м²",6/17 этаж,6980000,194429,NaN,"Республика Татарстан, Казань, р-н Советский, у...","вчера (вчера, 21:40)",Агентство недвижимости,ЭТАЖИ КАЗАНЬ
4,https://kazan.cian.ru/sale/flat/279798485/,"2-комн. кв., 53 м², 6/9 этаж",2-комн. кв.,53 м²,6/9 этаж,6500000,122642,NaN,"Республика Татарстан, Казань, р-н Вахитовский,...","2 недели назад (25 мар, 11:22)",NaN,ID 97030797
...,...,...,...,...,...,...,...,...,...,...,...,...
67854,https://kazan.cian.ru/sale/flat/284260313/,"2-комн. кв., 55,5 м², 3/10 этаж",2-комн. кв.,"55,5 м²",3/10 этаж,6990000,125946,NaN,"Республика Татарстан, Казань, р-н Советский, м...","пять дней назад (5 апр, 00:16)",Агентство недвижимости,ЭТАЖИ КАЗАНЬ
67855,https://kazan.cian.ru/sale/flat/284252006/,"3-комн. кв., 71 м², 1/10 этаж",3-комн. кв.,71 м²,1/10 этаж,6800000,95775,NaN,"Республика Татарстан, Казань, р-н Советский, м...","три дня назад (6 апр, 20:36)",NaN,ID 45399873
67856,https://kazan.cian.ru/sale/flat/284241806/,"2-комн. кв., 48,5 м², 3/19 этаж",2-комн. кв.,"48,5 м²",3/19 этаж,6700000,138144,NaN,"Республика Татарстан, Казань, р-н Советский, у...","4 часа назад (сегодня, 13:17)",Агентство недвижимости,Реневада
67857,https://kazan.cian.ru/sale/flat/284240049/,"2-комн. кв., 49,8 м², 2/9 этаж",2-комн. кв.,"49,8 м²",2/9 этаж,6900000,138554,NaN,"Республика Татарстан, Казань, р-н Ново-Савинов...","четыре дня назад (6 апр, 08:45)",Агентство недвижимости,Ключ


In [47]:
ilya_c['Адрес'].apply(lambda x: x.split(', ')[1]).value_counts().index

Index(['Краснодар', 'Казань', 'Анапа', 'Новороссийск', 'Челябинск', 'Томск',
       'Армавир', 'Анапа муниципальное образование', 'Набережные Челны',
       'Альметьевск', 'Динская ст-ца'],
      dtype='object')

In [17]:
def create_realty_1(df, fname, sql_engine, source, dict_realty_type=dict_realty_cian_avito):
    try:
        # new table
        cian_realty = pd.DataFrame()

        # сбор данных из таблиц в бд
        city_df, source_df, exc_code = get_tables_info(sql_engine)

        # удаление дубликатов
        df.drop_duplicates(subset=['Ссылка'], keep='last', inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        # add source
        df['source'] = df['Ссылка'].apply(lambda x: x.split('.')[1])

        # source_id
        source_df_dict = source_df.set_index('id').to_dict()['name']
        cian_realty['source_id'] = df['source'].apply(lambda x: source_id_from_link(x, source_df_dict, 'cian'))

        # ad_id
        cian_realty['ad_id'] = df['Ссылка'].apply(lambda x: x.split('/')[-2])
        
        # city
        df['city'] = df['Адрес'].apply(lambda x: x.split(', ')[1])

        # city_id
        city_df_dict = city_df.set_index('id').to_dict()['name']
        cian_realty['city_id'] = df['city'].apply(lambda x: city_id_from_link(x, city_df_dict))

        # удалить записи где city_id пустой (записи городов, которые не присутствуют в списке городов для парсинга)
        # ind_to_drop = cian_realty[(cian_realty['city_id'].isna())].index.tolist()
        # cian_realty.drop(ind_to_drop, inplace=True)
        # df.drop(ind_to_drop, inplace=True)
        # cian_realty.city_id = cian_realty.city_id.astype(int)

        # type_id

        cian_realty['type_id'] = df['Комнат'].apply(lambda x: type_id_from_room(x, dict_realty_type))
        # удалить записи аукционов и других объяв где не опредлен type_id
        ind_to_drop = cian_realty[(cian_realty['type_id'].isna())].index.tolist()
        cian_realty.drop(ind_to_drop, inplace=True)
        df.drop(ind_to_drop, inplace=True)
        cian_realty.type_id = cian_realty.type_id.astype(int)

        # addr
        cian_realty['addr'] = df['Адрес'].apply(lambda x: x.replace(',',';'))

        # district_id после addr потому что в нем используется адрес
        all_districts = get_districts(sql_engine)
        cian_realty['district_id'] = cian_realty.apply(
            lambda row: district_from_rn_mkrn(row, all_districts, sql_engine), axis=1)

        # square
        cian_realty['square'] = df['Площадь']

        # floor
        cian_realty['floor'], cian_realty['house_floors'] = df['floor'], df['house_floors']

        # link
        cian_realty['link'] = df['link']

        # date
        file_date = get_date_from_name(fname)
        cian_realty['date'] = file_date

        # status
        cian_realty['status'] = 0

        # version
        current_source = cian_realty.source_id.unique()[0]
        current_date = datetime.now().date()
        last_version = get_version_db(sql_engine, current_date, current_source)[0].iloc[0, 0]
        if last_version == None:
            current_version = 1
        else:
            current_version = last_version + 1

        cian_realty['version'] = current_version

        # offer_from
        cian_realty['offer_from'] = df['offer_from']

        # status_new
        cian_realty['status_new'] = 1

        # add fields of price and price_sqrm
        cian_realty['price'], cian_realty['price_sqrm'] = df['price'], df['price_sqrm']

        # delete rows where one of important values isna
        cian_realty_check = cian_realty[['source_id', 'ad_id', 'type_id', 'addr', 'link', 'date', 'version']]
        values_na_ind = cian_realty_check[cian_realty_check.isna().any(axis=1)].index
        if len(values_na_ind) > 0:
            cian_realty = cian_realty.drop(values_na_ind)

        cian_realty_to_return = cian_realty[['source_id', 'ad_id', 'city_id', 'district_id', 'type_id',
                                             'addr', 'square', 'floor','house_floors', 'link', 'date',
                                             'status', 'version', 'offer_from', 'status_new', 'price', 'price_sqrm']]
    except Exception as ex:
        print(traceback.format_exc())
        cian_realty_to_return = pd.DataFrame()
        file_date = None
    return cian_realty_to_return, file_date

In [16]:
def process_realty_1(local_dir, file_to_process, sql_engine, source):
    saved_files = get_local_files(local_dir)
    realty_df = pd.DataFrame()
    file_date = None
    for f_ind in range(len(saved_files)):
        if Path(saved_files[f_ind]).name == file_to_process:
            try:
                if source == 'avito':
                    saved_file = pd.read_csv(saved_files[f_ind], delimiter=';', encoding='utf8')
                    realty_df, file_date = create_realty_1(saved_file, Path(saved_files[f_ind]).stem,
                                                         sql_engine, 'avito')
                elif source == 'cian':
                    saved_file = pd.read_csv(saved_files[f_ind], delimiter=',', encoding='cp1251')
                    realty_df, file_date = create_realty_1(saved_file, Path(saved_files[f_ind]).stem,
                                                         sql_engine, 'cian')
            except Exception as ex:
                print(traceback.format_exc())
                print('Не удалось прочитать файл из {}: '.format(source), Path(saved_files[f_ind]))
        else:
            pass
    if len(realty_df) != 0:
        print('Успешно обработан файл {} для realty'.format(file_to_process))
        error_status = False
        return realty_df, file_date, error_status
    else:
        error_status = True
        print('Файл {} не удалось обнаружить для realty'.format(file_to_process))
        return realty_df, file_date, error_status

In [27]:
for filename in files_to_process_cian:
    # обработка realty циан
    print('обработка файла {} для cian'.format(filename))
    df_cian_realty, file_date, error_file_processing = process_realty(local_save_dir_cian, '27-03-23.csv',
                                                                      sql_engine, 'cian')

    # проверка состояния
    if error_file_processing:
        print('Ошибка при обработке файлов из CIAN {}. Перезапуск скрипта...'.format(filename))
        error_processing_files = True
        break
    else:
        error_processing_files = False
        print('Выгрузка в таблицу realty обработанного файла из cian:', filename)

    # выгрузка и обновление данных в таблице realty
#     error_create_temp_realty, error_getting_ad_id, error_loading_into_realty, error_updating_realty = \
#         load_and_update_realty_db(sql_engine, df_cian_realty, 'cian')

#     if any([error_create_temp_realty, error_getting_ad_id,
#             error_loading_into_realty,error_updating_realty]):
#         close_sql_connection(sql_server, sql_engine)
#         print('Ошибка:', show_error([error_create_temp_realty, error_getting_ad_id,
#                                      error_loading_into_realty, error_updating_realty]))
#         print('Не удалось добавить данные в таблицу {}. Перезапуск скрипта...'.format('realty'))
#         time.sleep(10)
#         break

обработка файла циан 10-04-23 (без дублей).csv для cian
Успешно обработан файл 27-03-23.csv для realty
Выгрузка в таблицу realty обработанного файла из cian: циан 10-04-23 (без дублей).csv


In [28]:
df_cian_realty.head(5)

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,price,price_sqrm
0,2,210197059,5.0,NaN,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-27,0,287,Собственник,1,24000000,224299
1,2,213575526,5.0,NaN,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-27,0,287,Риелтор,1,10100000,95283
2,2,232012292,5.0,NaN,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-27,0,287,Риелтор,1,2100000,105000
3,2,233965925,5.0,NaN,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-27,0,287,Агентство недвижимости,1,4500000,155172
4,2,239179225,5.0,NaN,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-27,0,287,Собственник,1,15300000,109286


In [18]:
error_create_temp_realty, error_getting_ad_id, error_loading_into_realty, error_updating_realty = \
        load_and_update_realty_db(sql_engine, df_cian_realty, 'cian')

Временная таблица очищена
Загрузка новых объявлений в таблицу
новые объявления добавлены, переход к обновлению существующих
new_algo2
Обновление данных завершено


In [19]:
error_create_temp_realty, error_getting_ad_id, error_loading_into_realty, error_updating_realty

(False, False, False, False)

In [16]:
for filename in files_to_process_cian:
    # обработка realty циан
    print('обработка файла {} для cian'.format(filename))
    df_cian_realty, file_date, error_file_processing = process_realty(local_save_dir_cian, filename,
                                                                      sql_engine, 'cian')

    # проверка состояния
    if error_file_processing:
        print('Ошибка при обработке файлов из CIAN {}. Перезапуск скрипта...'.format(filename))
        error_processing_files = True
        break
    else:
        error_processing_files = False
        print('Выгрузка в таблицу realty обработанного файла из cian:', filename)

    # загрузка realty в таблицу
    try:
#         df_cian_realty[list_realty_cols].to_sql(name='realty', con=sql_engine, if_exists='append',
#                                chunksize=7000, method='multi', index=False)
        # df_cian_realty.to_csv(f'{filename}_test_cian_realty.csv')
        error_writing_files = False
    except Exception as exc:
        print(exc)
        print('Не удалось добавить данные в таблицу {}. Перезапуск скрипта...'.format('realty'))
        error_writing_files = True
        break

обработка файла 27-03-23.csv для cian
Успешно обработан файл 27-03-23.csv для realty
Выгрузка в таблицу realty обработанного файла из cian: 27-03-23.csv


# апдейт

In [18]:
exist_ad_id = get_exist_ad_id(sql_engine, 'cian')[0].ad_id.to_list()

In [36]:
df_cian_realty_exist___ = df_cian_realty[df_cian_realty.ad_id.isin([])][list_realty_cols]

In [38]:
len(df_cian_realty_exist___)

0

In [120]:
df_cian_realty_new

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
1189,2,270328443,5.0,NaN,6,Краснодарский край; Анапа; Владимирская улица;...,95.0,20,20,https://anapa.cian.ru/sale/flat/270328443/,2023-03-27,None,274,NaN,1
3306,2,278996199,5.0,NaN,3,Краснодарский край; Анапа; Алексеевка мкр; ули...,35.0,6,7,https://anapa.cian.ru/sale/flat/278996199/,2023-03-27,None,274,Собственник,1
3433,2,279241124,5.0,NaN,2,Краснодарский край; Анапа; Таманская улица; 12...,26.0,5,6,https://anapa.cian.ru/sale/flat/279241124/,2023-03-27,None,274,Агентство недвижимости,1
3622,2,279601823,5.0,NaN,3,Краснодарский край; Анапа; улица Толстого; 26,35.5,2,7,https://anapa.cian.ru/sale/flat/279601823/,2023-03-27,None,274,Представитель застройщика,1
4159,2,280802412,5.0,NaN,3,Краснодарский край; Анапа; 18/3 ЖК,44.7,7,15,https://anapa.cian.ru/sale/flat/280802412/,2023-03-27,None,274,Застройщик,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103599,2,285370017,17.0,387.0,4,Томская область; Томск; р-н Кировский; д. Лоск...,57.1,2,4,https://tomsk.cian.ru/sale/flat/285370017/,2023-03-27,None,274,NaN,1
103600,2,285370096,17.0,405.0,4,Томская область; Томск; р-н Октябрьский; мкр. ...,53.0,8,10,https://tomsk.cian.ru/sale/flat/285370096/,2023-03-27,None,274,Агентство недвижимости,1
103601,2,285370463,17.0,417.0,4,Томская область; Томск; р-н Ленинский; мкр. Ра...,53.0,6,10,https://tomsk.cian.ru/sale/flat/285370463/,2023-03-27,None,274,Агентство недвижимости,1
103602,2,285373005,17.0,394.0,3,Томская область; Томск; р-н Ленинский; Черемош...,31.5,7,10,https://tomsk.cian.ru/sale/flat/285373005/,2023-03-27,None,274,Агентство недвижимости,1


In [19]:
df_cian_realty_exist = df_cian_realty[df_cian_realty.ad_id.isin(exist_ad_id)][list_realty_cols]
df_cian_realty_new = df_cian_realty[~df_cian_realty.ad_id.isin(exist_ad_id)][list_realty_cols]

In [24]:
df_cian_realty_exist.head(3)

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5.0,NaN,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-27,None,274,Собственник,1
1,2,213575526,5.0,NaN,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-27,None,274,Риелтор,1
2,2,232012292,5.0,NaN,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-27,None,274,Риелтор,1


In [ ]:
# обновление данных
df_cian_realty_exist.to_sql(name='temp_realty', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)

con_obj = sql_engine.connect()
common_ids = tuple(df_cian_realty_exist.ad_id)
update_table_query = f"""update realty join temp_realty on realty.ad_id=temp_realty.ad_id
                        set realty.source_id = temp_realty.source_id,
                            realty.city_id = temp_realty.city_id,
                            realty.district_id = temp_realty.district_id,
                            realty.type_id = temp_realty.type_id,
                            realty.addr = temp_realty.addr,
                            realty.square = temp_realty.square,
                            realty.floor = temp_realty.floor,
                            realty.house_floors = temp_realty.house_floors,
                            realty.link = temp_realty.link,
                            realty.date = temp_realty.date,
                            realty.status = temp_realty.status,
                            realty.version = temp_realty.version,
                            realty.offer_from = temp_realty.offer_from,
                            realty.status_new = temp_realty.status_new,
                            realty.house_id = temp_realty.house_id
                         WHERE realty.ad_id in {common_ids}"""
con_obj.execute(text(update_table_query))
con_obj.commit()
con_obj.close()

In [ ]:
# очистка данных из temp_realty
con_obj = sql_engine.connect()
con_obj.execute(text(clear_temp_table_query))
con_obj.commit()
con_obj.close()

In [ ]:
# добавление данных 
df_cian_realty_new.to_sql(name='realty', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)